In [1]:
pip install Pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=36580ef203c38cb390f5071268922f8c9060bc69d96ebf2e566c50eb3a709f0b
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built Pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('NLP').getOrCreate()

In [7]:
data = spark.read.csv('SMSSpamCollection',inferSchema =True,sep='\t' )
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [8]:
#renaming column headers
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [9]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import length

In [11]:
data = data.withColumn('length',length(data['text']))

In [12]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [13]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [17]:
### Feature transformations
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [44]:
tokenizer=Tokenizer(inputCol='text',outputCol = 'token_text')
stop_remove =StopWordsRemover(inputCol='token_text',outputCol = 'stop_tokens')
count_vec = CountVectorizer(inputCol = 'stop_tokens',outputCol = 'c_vec')
idf = IDF(inputCol='c_vec',outputCol ="tf-idf")
ham_spam_num = StringIndexer(inputCol='class',outputCol= 'label')

In [45]:
from pyspark.ml.feature import VectorAssembler

In [46]:
clean_data = VectorAssembler(inputCols=['tf-idf','length'],outputCol='features')

In [47]:
from pyspark.ml.classification import   NaiveBayes

In [48]:
nb= NaiveBayes()

In [49]:
#creating a pipeline
from pyspark.ml import Pipeline
data_pipeline = Pipeline(stages=[ham_spam_num,tokenizer,stop_remove,count_vec,idf,clean_data])

In [50]:
cleaned_data =data_pipeline.fit(data)

In [51]:
use_data =cleaned_data.transform(data)

In [53]:

use_data = use_data.select(['label','features'])
use_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [54]:
#split the data to train and test

train_data,test_data = use_data.randomSplit([0.8,0.2])

In [55]:
spam_predictor = nb.fit(train_data)

In [57]:
test_results = spam_predictor.transform(test_data)

In [58]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,11,32...|[-870.52096378121...|[1.0,3.8539942315...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-442.06209958453...|[1.0,3.6381952821...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-694.46641364281...|[1.0,2.3930887574...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1015.0741658846...|[1.0,5.5810260184...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1301.3335624219...|[1.0,3.1179887826...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-1496.5701027401...|[0.99999432364669...|       0.0|
|  0.0|(13424,[0,1,30,11...|[-612.00675815439...|[1.0,2.2448243411...|       0.0|
|  0.0|(13424,[0,1,71,10...|[-675.78195333716...|[1.0,1.1803181417...|       0.0|
|  0.0|(13424,[0,1,150,1...|[-256.81391789962...|[0.02138861144006...|       1.0|
|  0.0|(13424,[0

In [59]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [60]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9280082661195318
